In [ ]:
%pip install lm-eval

In [1]:
import torch
import lm_eval
import pandas as pd
from lm_eval.models.huggingface import HFLM
from IPython.display import display, HTML
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
device = torch.device('cuda:3')

In [3]:
model_name = 'microsoft/phi-2' 

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((256

In [4]:
lm_obj = HFLM(pretrained=model, tokenizer=tokenizer)
results = lm_eval.simple_evaluate(
    model=lm_obj,
    tasks=["truthfulqa"],
    num_fewshot=0,
    limit=10)

2024-04-27:06:37:49,319 WARNING  [huggingface.py:118] `pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
2024-04-27:06:37:49,336 WARNING  [huggingface.py:337] Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
2024-04-27:06:37:49,338 INFO     [evaluator.py:131] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-04-27:06:38:01,369 INFO     [evaluator.py:218] num_fewshot has been set to 0 for truthfulqa_mc1 in its config. Manual configuration will be ignored.
2024-04-27:06:38:01,370 INFO     [evaluator.py:218] num_fewshot has been set to 0 for truthfulqa_mc2 in its config. Manual configuration will be ignored.
2024-04-27:06:38:01,371 INFO     [evaluator.py:218] num_fewshot has been set to 0 for truthfulqa_gen in its config. 

In [5]:
df = pd.DataFrame(results['results']).transpose()
df['Baseline Accuracy'] = df[['acc,none','acc_stderr,none']].apply(lambda x : '{} ± {}'.format(round(x[0],2), round(x[1], 4)), axis=1)
df[['Baseline Accuracy']]

/tmp/ipykernel_3667617/3071663309.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['Baseline Accuracy'] = df[['acc,none','acc_stderr,none']].apply(lambda x : '{} ± {}'.format(round(x[0],2), round(x[1], 4)), axis=1)


,Baseline Accuracy
truthfulqa,0.51 ± 0.1097
truthfulqa_gen,nan ± nan
truthfulqa_mc1,0.4 ± 0.1633
truthfulqa_mc2,0.61 ± 0.1465


In [6]:
from typing import Optional, Union, Tuple, List

from torch.nn import CrossEntropyLoss

from transformers.models.phi.modeling_phi import PhiForCausalLM, PhiConfig
from transformers.modeling_outputs import CausalLMOutputWithPast
import torch.nn.functional as F


def jensen_shannon_divergence(p, q, base=2.0):
    p = F.softmax(p, dim=-1)
    q = F.softmax(q, dim=-1)

    m = 0.5 * (p + q)

    kl_pm = F.kl_div(p.log(), m, reduction='sum')
    kl_qm = F.kl_div(q.log(), m, reduction='sum')

    jsd = 0.5 * (kl_pm + kl_qm)

    if base != 2.0:
        jsd = jsd / torch.log(torch.tensor(base))

    return jsd

# Some code snippets were copied from transformers.models.phi.modeling_phi.PhiForCausalLM.forward()
class PhiDoLaPoE(PhiForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.final_layernorm = torch.nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)

    def forward(
            self,
            input_ids: torch.LongTensor = None,
            attention_mask: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.LongTensor] = None,
            past_key_values: Optional[List[torch.FloatTensor]] = None,
            inputs_embeds: Optional[torch.FloatTensor] = None,
            labels: Optional[torch.LongTensor] = None,
            use_cache: Optional[bool] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
        ) -> Union[Tuple, CausalLMOutputWithPast]:

            output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
            output_hidden_states = True
            return_dict = return_dict if return_dict is not None else self.config.use_return_dict

            # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                inputs_embeds=inputs_embeds,
                use_cache=use_cache,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )

            hidden_states = outputs[0]
            logits = hidden_states 
            logits = self.lm_head(hidden_states)
            logits = logits.float()

            intermediate_outputs = ()

            max_jsd = float('-inf')
            max_logits = None

            for hidden_state in outputs.hidden_states[:-1]:
                temp = self.lm_head(self.final_layernorm(hidden_state)).float()
                intermediate_outputs += (temp,)
                curr_jsd = jensen_shannon_divergence(logits[-1], temp[-1])

                if curr_jsd > max_jsd:
                    max_jsd = curr_jsd
                    max_logits = temp

            logits[-1] -= max_logits[-1]

            loss = None
            if labels is not None:
                # Shift so that tokens < n predict n
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()
                # Flatten the tokens
                loss_fct = CrossEntropyLoss()
                shift_logits = shift_logits.view(-1, self.config.vocab_size)
                shift_labels = shift_labels.view(-1)
                # Enable model parallelism
                shift_labels = shift_labels.to(shift_logits.device)
                loss = loss_fct(shift_logits, shift_labels)

            if not return_dict:
                output = (logits,) + outputs[1:]
                return (loss,) + output if loss is not None else output

            return CausalLMOutputWithPast(
                loss=loss,
                logits=logits,
                past_key_values=outputs.past_key_values,
                hidden_states=intermediate_outputs,
                attentions=outputs.attentions,
            )

In [7]:
poe_model = PhiDoLaPoE.from_pretrained('microsoft/phi-2')
poe_model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
lm_obj = HFLM(pretrained=poe_model, tokenizer=tokenizer)
results = lm_eval.simple_evaluate(
    model=lm_obj,
    tasks=["truthfulqa"],
    num_fewshot=0,
    limit=10)

2024-04-27:04:46:19,371 WARNING  [huggingface.py:118] `pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
2024-04-27:04:46:19,386 WARNING  [huggingface.py:337] Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
2024-04-27:04:46:19,388 INFO     [evaluator.py:131] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-04-27:04:46:28,879 INFO     [evaluator.py:218] num_fewshot has been set to 0 for truthfulqa_mc1 in its config. Manual configuration will be ignored.
2024-04-27:04:46:28,880 INFO     [evaluator.py:218] num_fewshot has been set to 0 for truthfulqa_mc2 in its config. Manual configuration will be ignored.
2024-04-27:04:46:28,881 INFO     [evaluator.py:218] num_fewshot has been set to 0 for truthfulqa_gen in its config. 

In [19]:
df_poe= pd.DataFrame(results['results']).transpose()
df['PoE Accuracy'] = df_poe[['acc,none','acc_stderr,none']].apply(lambda x : '{} ± {}'.format(round(x[0],2), round(x[1], 4)), axis=1)
df[['Baseline Accuracy','PoE Accuracy' ]]

/tmp/ipykernel_3651612/784578564.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['PoE Accuracy'] = df_poe[['acc,none','acc_stderr,none']].apply(lambda x : '{} ± {}'.format(round(x[0],2), round(x[1], 4)), axis=1)


,Baseline Accuracy,PoE Accuracy
truthfulqa,0.51 ± 0.1097,0.59 ± 0.1078
truthfulqa_gen,nan ± nan,nan ± nan
truthfulqa_mc1,0.4 ± 0.1633,0.5 ± 0.1667
truthfulqa_mc2,0.61 ± 0.1465,0.67 ± 0.1368
